In [6]:
import pandas as pd
import os
import csv
from google.colab import drive
import math
from collections import Counter, defaultdict
import os

In [8]:
drive.mount('/content/drive')

fb_posts = '/content/drive/My Drive/Presidential Election Data/2024_fb_posts_president_scored_anon.csv'
fb_ads = '/content/drive/My Drive/Presidential Election Data/2024_fb_ads_president_scored_anon.csv'
tw_posts = '/content/drive/My Drive/Presidential Election Data/2024_tw_posts_president_scored_anon.csv'

Mounted at /content/drive


In [3]:
def analyze_dataset(df):
    print("===== General Dataset Summary =====")

    for col in df.columns:
        print(f"\nColumn: {col}")
        col_data = df[col].dropna()

        if pd.api.types.is_numeric_dtype(col_data):
            print(f"  Count: {col_data.count()}")
            print(f"  Mean: {col_data.mean()}")
            print(f"  Min: {col_data.min()}")
            print(f"  Max: {col_data.max()}")
            print(f"  Stddev: {col_data.std()}")
        else:
            print(f"  Count: {col_data.count()}")
            print(f"  Unique values: {col_data.nunique()}")
            if not col_data.empty:
                most_freq = col_data.mode().iloc[0]
                freq_count = col_data.value_counts().iloc[0]
                print(f"  Most frequent: {most_freq} (appears {freq_count} times)")


In [9]:
df_ads = pd.read_csv(fb_ads)
df_ads = df_ads.replace(' ', pd.NA)  # Treat empty strings as NA
# Convert numeric columns automatically
df_ads = df_ads.apply(pd.to_numeric, errors='ignore')
analyze_dataset(df_ads)

/tmp/ipython-input-9-3796498448.py:4: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_ads = df_ads.apply(pd.to_numeric, errors='ignore')


===== General Dataset Summary =====

Column: page_id
  Count: 246745
  Unique values: 4475
  Most frequent: 4d66f5853f0365dba032a87704a634f023d15babde973bb7a284ed8cd2707b2d (appears 55503 times)

Column: ad_id
  Count: 246745
  Unique values: 246745
  Most frequent: 0000a88a64484883df6ca1fabd357e4c5950e443123ce3f45d1c5b4c621f9783 (appears 1 times)

Column: ad_creation_time
  Count: 246745
  Unique values: 547
  Most frequent: 2024-10-27 (appears 8619 times)

Column: bylines
  Count: 245736
  Unique values: 3790
  Most frequent: HARRIS FOR PRESIDENT (appears 49788 times)

Column: currency
  Count: 246745
  Unique values: 18
  Most frequent: USD (appears 246599 times)

Column: delivery_by_region
  Count: 246745
  Unique values: 141122
  Most frequent: {} (appears 30989 times)

Column: demographic_distribution
  Count: 246745
  Unique values: 215622
  Most frequent: {} (appears 30989 times)

Column: estimated_audience_size
  Count: 246745
  Mean: 556462.8559687126
  Min: 0
  Max: 1000001


## Grouped Aggregation

In [14]:
import pandas as pd

def analyze_grouped_dataset(df, group_cols):
    print(f"\n===== Grouped Analysis by {group_cols} =====")

    grouped = df.groupby(group_cols)

    for i, (group_key, group_df) in enumerate(grouped):
        if i >= 3:
            break  # Limit to first 3 groups

        print(f"\nGroup: {group_key}")
        for col in group_df.columns:
            if col in group_cols:
                continue  # Skip group keys

            col_data = group_df[col].dropna()
            print(f"  Column: {col}")

            if pd.api.types.is_numeric_dtype(col_data):
                print(f"    Count: {col_data.count()}")
                print(f"    Mean: {col_data.mean()}")
                print(f"    Min: {col_data.min()}")
                print(f"    Max: {col_data.max()}")
                print(f"    Stddev: {col_data.std()}")
            else:
                print(f"    Count: {col_data.count()}")
                print(f"    Unique values: {col_data.nunique()}")
                if not col_data.empty:
                    most_freq = col_data.mode().iloc[0]
                    freq_count = col_data.value_counts().iloc[0]
                    print(f"    Most frequent: {most_freq} (appears {freq_count} times)")

In [15]:
analyze_grouped_dataset(df_ads, ['page_id'])
analyze_grouped_dataset(df_ads, ['page_id', 'ad_id'])


===== Grouped Analysis by ['page_id'] =====

Group: ('0005687f4ac876c135dda2bd5aa60962eced3356228ef01c21c514d263a11128',)
  Column: ad_id
    Count: 2
    Unique values: 2
    Most frequent: 10913e6cae50d9aafe81b85c23f4ad11e24bde5d4f24643c14734a19a2733e5e (appears 1 times)
  Column: ad_creation_time
    Count: 2
    Unique values: 2
    Most frequent: 2024-08-23 (appears 1 times)
  Column: bylines
    Count: 2
    Unique values: 1
    Most frequent: David Blount for State Senate (appears 2 times)
  Column: currency
    Count: 2
    Unique values: 1
    Most frequent: USD (appears 2 times)
  Column: delivery_by_region
    Count: 2
    Unique values: 2
    Most frequent: {'Alabama': {'spend': 0, 'impressions': 26}, 'Pennsylvania': {'spend': 0, 'impressions': 3}, 'New Hampshire': {'spend': 0, 'impressions': 1}, 'New Jersey': {'spend': 0, 'impressions': 2}, 'New York': {'spend': 0, 'impressions': 21}, 'North Carolina': {'spend': 0, 'impressions': 23}, 'Ohio': {'spend': 0, 'impressions': 1